<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Intro" data-toc-modified-id="Intro-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Intro</a></span></li><li><span><a href="#Implementation" data-toc-modified-id="Implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Implementation</a></span></li><li><span><a href="#Example" data-toc-modified-id="Example-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Example</a></span></li></ul></div>

# Command

## Intro

The Command design pattern encapsulates a request as an object. This lets you parametise clients with different requests.

The command object binds together a set of actions on a specific receiver. It puts these behind a single method. normally called execute. You can also store state for reversing actions as part of execute, essentially supporting an undo functionality.

We should also implement a logging as part of the pattern. With this, it is very useful for modeling transactions.

Some good examples of this pattern are orders to buy and sell a thing. The orders themselves are a command. Each order is the same, so can be encapsulated in a package.

We should use this pattern when we want to paratemise objects by an action to perform. That is, we want objects in context doing the thing, but not containing the logic of the thing. As in the example above, an order is placed by someone, that person could want a different order. The _process_ of the order is the same, but the context can change.

It could also be used when we want to queue or execute requests in sequence. Because the objects can be passed around, they can be stored and batched correctly and efficiently.

It's also useful when we need to have transactional requests. That is something that can be exchanged and reversed.

The constituent parts of the pattern are:
> Client - creates a concrete command object and sets its receiver

> Invoker - asks the command to carry out the request (calls the execute method)

> Receiver - Knows how to perform the work needed to carry out a request

## Implementation

In [11]:
interface Receiver {
     void openFile();
     void writeFile();
     void closeFile();
 }

In [ ]:
interface Command {
    void execute();
}

In [19]:
interface IInvoker {
    void execute();
}

In [13]:
class LinuxReceiver implements Receiver {
    public void openFile()
    {
        System.out.println("Opening file");
    }

    public void writeFile()
    {
        System.out.println("Writing file");
    }
    
    public void closeFile()
    {
        System.out.println("Close file");
    }
}

In [15]:
class OpenCommand implements Command {
    private Receiver receiver;
    
    // Can store previous state inside here with member vars
    
    public OpenCommand(Receiver rec)
    {
        this.receiver = rec;
    }
    
    public void execute()
    {
        this.receiver.openFile();
    }
}

// Only implementing one command here on purpose

In [20]:
class Invoker implements IInvoker {
    public Command command;
    
    public Invoker(Command com)
    {
        this.command = com;
    }
    
    public void execute()
    {
        this.command.execute();
    }
}

In [21]:
Receiver myReceiver = new LinuxReceiver();
Command myCommand = new OpenCommand(myReceiver);
Invoker myInvoker = new Invoker(myCommand); 

In [22]:
myInvoker.execute();

Opening file


This pattern is actually kinda simple when we cut through the OO bullshit. The invoker does what it sounds like it does. It kicks off the process by executing the command. 

The command then does its thing (because the command is specific to the effect we want to have, it's implementation can vary). The receiver is the thing that does the thing the command references in context.

This example is of a dumb command (it doesn't do anything other than delegate to the receiver.)

## Example

In [32]:
interface Order {
    void execute();
    void undo();
}

In [33]:
interface Stock {
    void buy(int amount);
    void sell(int amount);
}

In [52]:
interface Agent {
    void submit(Order order);
    void reverse(Order order);
}

In [53]:
class BuyOrder implements Order {
    private Stock stock;
    private int amount;
    
    public BuyOrder(Stock stock, int amount)
    {
        this.stock = stock;
        this.amount = amount;
    }
    
    public void execute()
    {
        this.stock.buy(this.amount);
    }
    
    public void undo()
    {
        this.stock.sell(this.amount);
    }
}

In [54]:
class SellOrder implements Order {
    private Stock stock;
    private int amount;
    
    public SellOrder(Stock stock, int amount)
    {
        this.stock = stock;
        this.amount = amount;
    }
    
    public void execute()
    {
        this.stock.sell(this.amount);
    }
    
    public void undo()
    {
        this.stock.buy(this.amount);
    }
}

In [55]:
class JustPark implements Stock {
    public void buy(int amount)
    {
        System.out.println("Buying £" + amount + " of JustPark stock");
    }
    
    public void sell(int amount)
    {
        System.out.println("Selling £" + amount + " of JustPark stock");
    }
}

In [56]:
class MyAgent implements Agent {
    public void submit(Order order)
    {
        order.execute();
    }
    
    public void reverse(Order order)
    {
        order.undo();
    }
}

In [57]:
Stock justpark = new JustPark();

In [58]:
Order buy = new BuyOrder(justpark, 25);

In [59]:
Agent myAgent = new MyAgent();

In [60]:
myAgent.submit(buy);

Buying £25 of JustPark stock


In [61]:
myAgent.reverse(buy);

Selling £25 of JustPark stock
